## Programmation vectorielle

Le but des exercices est

* d'avoir un programme qui donne la bonne réponse
* qui soit le plus rapide possible (et pour cela on utilise massivement Numpy)

En règle général si vous avez des `for` imbriqués c'est mauvais signe.

In [1]:
import numpy as np

np.set_printoptions(precision=10, linewidth=150, suppress=True)

## Méthode du pivot de Gauss partiel

L'ennoncé est dans le cours (et lire le cours peut servir). On verifiera sur le cas qui génère des
erreurs d'arrondis.

La structure de la matrice A après 3 itérations ressemble à cela :

```
 ----------------
  \              |
    \            |
      \          |
       |         |
   0   |         |
       |         |
       -----------
```
Aussi il est important que ne pas détruire le travail déjà fait à savoir de garder les 0 sous la diagonale.
Cela implique qu'on ne doit pas changer la 4e ligne avec une ligne au dessus. On ne regarde donc que les
valeurs de la 4e colonne qui sont en dessous de la diagonale pour choisir le nouveau pivot.

In [2]:
def solve_gauss_partial(A, b):   # on prend le max dans la colonne i parmi les lignes en dessous (plus facile)
    for i in range(len(A)-1):
        pivot = i + np.argmax(np.abs(A[i:, i]))  # il n'y a que 3 lignes à ajouter pour échanger les lignes
        A[[i, pivot]] = A[[pivot, i]]
        b[[i, pivot]] = b[[pivot, i]]
        # la suite est comme le pivot de Gauss normal
        coefs = - A[i+1:,i] / A[i,i]
        A[i+1:, i:] += np.outer(coefs, A[i, i:]) # ou np.einsum('i,j -> ij', coefs, A[i, i:])
        b[i+1:] += coefs * b[i]                  # multiplication terme à terme
    # A est maintenant triangulaire surpérieur
    res = np.zeros(len(b), dtype=b.dtype)
    res[-1] = b[-1] / A[-1,-1]
    for i in range(len(A)-1)[::-1]:
        res[i] = (b[i] - A[i,i+1:] @ res[i+1:]) / A[i,i]
    return res

In [3]:
e = 1E-6
A = np.array([[e, 1], [1, 2]], dtype='float32')
b = np.array([1., 3.], dtype='float32')
print(f"A\n {A} \nb\n {b}\n")
x = solve_gauss_partial(A, b)
print('solution : ',x)
print('vérification\n', A@x)

A
 [[0.000001 1.      ]
 [1.       2.      ]] 
b
 [1. 3.]

solution :  [1.0000019  0.99999905]
vérification
 [3.       0.999997]


## Factorisation de Choleski

Il s'agit de décomposer A en $A = B\, B^T$ avec B une matrice triangulaire inférieure. Cela n'est possible
que si la matrice A est symétrique et définie positive (c'est d'ailleurs une facon de vérifier qu'une
matrice est définie positive).

Écrire l'algorithme de Choleski qui prend A et retourne B (pour deviner l'algorithme, essayez de trouver les 
coefficients de B à partir des coefficients de A sur une matrice A 4x4).

**Calculs** A faire sur papier, c'est bien plus simple !

$$
A = B \, B^T = 
\begin{bmatrix}
b_{11} \; 0 \; \; \ldots 0 \\
b_{21} \; b_{22} \ldots 0 \\
 \vdots \\
b_{n1} b_{n2} \ldots b_{n,n} \\
\end{bmatrix}
\;
\begin{bmatrix}
b_{11} \; b_{21} \; \; \ldots b_{n1} \\
0 \; \; b_{22} \ldots \; b_{n2} \\
 \vdots \\
0 \; \; 0 \; \; \ldots \; b_{n,n} \\
\end{bmatrix}
=
\begin{bmatrix}
b_{11}^2 \quad b_{11} b_{21} \quad \ldots \quad b_{11} b_{n1} \\
x \; \; \sum_{i=1}^2 b_{2i}^2 \; \ldots \; \sum_{i=1}^2 b_{2i} b_{ni} \\
 \vdots \\
x \quad x \quad \ldots \qquad \sum_{i=1}^n b_{n,i}^2 \\
\end{bmatrix}
$$

avec $x$ la même valeur que de l'autre coté de la diagonale (je vous laisse vérifier et il le faut puisque A
est symétrique).

On voit tout de suite que $b_{11} = \sqrt{a_{11}}$ et maintenant qu'on a $b_{11}$ on peut trouver 
toute la première ligne de $B^T$ : $b_{j1} = a_{1j} / b_{11}$. 

Une fois qu'on connait la première ligne de $B^T$, on s'attaque à la deuxième en commencant par trouver
$b_{22}$ puis ensuite tous les autres éléments de la ligne comme on a fait pour la première ligne.

On a donc dans le cas général :

* $b_{ii} = \sqrt{a_{ii} -  \sum_{k=1}^{i-1} b_{ik}^2} $
* $\displaystyle b_{ji} = \frac{a_{ij} - \sum_{k=1}^{i-1} b_{ik} \, b_{jk}}{b_{ii}} = \frac{a_{ij} - \sum_{k=1}^{i-1} b_{ik} \, b^T_{kj}}{b_{ii}} \quad \forall j > i$

In [4]:
def Choleski(A):
    B = np.zeros(A.shape)
    for i in range(len(A)):
        B[i,i] = np.sqrt(A[i,i] - np.sum(np.square(B[i, :i])))         # garanti ok car A est def positive
        B[i+1:, i] = (A[i, i+1:] - B[i, :i] @ B.T[:i, i+1:]) / B[i,i]  # les ∑ sous forme de prod. scalaire
    return B

Rappel : pas de boucles `for` imbriquées mais des opérations vectorielles et matricielles (sur des sous-matrices).

Créer une matrice symétrique définie positive et vérifier que son programme marche bien.

In [5]:
A = np.random.randint(10, size=(4,4))
A = A + A.T                             # symmétrique
A = A + np.diag(A.sum(axis=0))          # diagonale dominante
print('A:\n', A)
B = Choleski(A)
print('B\n', B)
print('vérification\n', B @ B.T)

A:
 [[47 15 11 13]
 [15 72 14 15]
 [11 14 41 12]
 [13 15 12 76]]
B
 [[6.8556546004 0.           0.           0.          ]
 [2.1879748725 8.1983392195 0.           0.          ]
 [1.6045149065 1.2794495838 6.065355775  0.          ]
 [1.8962448895 1.323568535  1.1976226732 8.3197428797]]
vérification
 [[47. 15. 11. 13.]
 [15. 72. 14. 15.]
 [11. 14. 41. 12.]
 [13. 15. 12. 76.]]


## Amméliorer Jacobi

Lorsqu'on écrit une itération de la méthode de Jacobi avec l'ensemble des coefficients, on constate que
si on calcule la nouvelle valeur de **x** élément par élement alors lorsqu'on veut mettre à jour `x[1]`, 
on connait déjà `x[0]`. Idem lorsqu'on met à jour `x[2]` on connait déjà `x[0]` et `x[1]`, etc.

L'idée de la version amméliorée de Jacobi est d'utiliser la nouvelle valeur de `x[0]` et non pas l'ancienne
comme c'est le cas dans l'algorithme du cours. Ainsi en utilisant des valeurs mise à jour on peut espérer
converger plus vite.

Dans ce chaque itération demande un calcul ligne par ligne et donc une boucle `for` dans la boucle `for` sur
les itérations.

#### Test d'arrêt

On ajoutera un argument `error` à la fonction qui indique la précision désirée du résultat. Par
défaut sa valeur est de `1E-6` et pour offrir une bonne garantie on arrête l'algorithme lorsque
$||x_{t+1} - x_t|| < \texttt{error}\, / \, 10$.

In [6]:
def Jacobi(A, b, error=1E-6, verbose=False):  # version avec double boucle (for dans le while des itérations)
    M = np.diag(A)        # attention, c'est un vecteur
    N = np.diag(M) - A    # np.diag d'une matrice donne un vecteur, np.diag d'un vecteur donne une matrice
    x = np.random.random(len(b))
    previous_x = np.zeros(len(b))
    err = (error / 10) ** 2
    while np.sum(np.square(x - previous_x))  > err:
        previous_x = x.copy()
        if verbose:
            print(f"x = {x}")
        for i in range(len(x)):
            x[i] = (N[i] @ x + b[i]) / M[i]   # on utilise à droite le x modifié (au lieu de previous_x)
    return x

In [7]:
A = np.random.randint(10, size=(4,4))
A = A + np.diag(A.sum(axis=0))
b = A.sum(axis=1)                     # ainsi la solution est [1,1,1,1]
print('A:\n', A, "\nb:\n", b, "\n")

Jacobi(A,b, verbose=True)

A:
 [[27  2  1  9]
 [ 5 19  9  5]
 [ 0  6 11  1]
 [ 4  7  1 33]] 
b:
 [39 38 18 45] 

x = [0.4343574573 0.7716781542 0.4717712717 0.3566743709]
x = [1.2509186327 1.3534786072 0.8656776351 0.8986756844]
x = [1.0125660738 1.0869838155 0.9617655839 0.9811843458]
x = [1.0012447286 1.022734967  0.9893096229 0.9953505059]
x = [1.0002616996 1.0062185456 0.9970307473 0.9987391708]
x = [1.0000696157 1.0017199653 0.9991764579 0.9996516764]
x = [1.0000192046 1.0004767091 0.9997716426 0.999903472 ]
x = [1.0000053219 1.0001321709 0.9999366821 0.9999732374]
x = [1.0000014755 1.0000366472 0.9999824436 0.9999925795]
x = [1.0000004091 1.0000101613 0.9999951321 0.9999979425]
x = [1.0000001134 1.0000028175 0.9999986502 0.9999994295]
x = [1.0000000315 1.0000007812 0.9999996257 0.9999998418]
x = [1.0000000087 1.0000002166 0.9999998962 0.9999999561]
x = [1.0000000024 1.0000000601 0.9999999712 0.9999999878]


array([1.0000000007, 1.0000000167, 0.999999992 , 0.9999999966])

La méthode de base de Jacobi sur le même système matriciel prend nettement plus d'itérations :

In [8]:
M = np.diag(A)        # attention, c'est un vecteur
N = np.diag(M) - A    # np.diag d'une matrice donne un vecteur, np.diag d'un vecteur donne une matrice
print(f"M:\n {np.diag(M)}\nN:\n {N}\n")

x = np.random.random(4)
previous_x = np.zeros(len(b))
err = (1E-6 / 10) ** 2
while np.sum(np.square(x - previous_x))  > err:
    previous_x = x.copy()
    print(f"x = {x}")
    x = (N @ x + b) / M

M:
 [[27  0  0  0]
 [ 0 19  0  0]
 [ 0  0 11  0]
 [ 0  0  0 33]]
N:
 [[ 0 -2 -1 -9]
 [-5  0 -9 -5]
 [ 0 -6  0 -1]
 [-4 -7 -1  0]]

x = [0.3954247911 0.2991827077 0.067341483  0.5044888257]
x = [1.2516256915 1.7312820299 1.427310448  1.2502024363]
x = [0.8466038358 0.6655297542 0.5783732167 0.8014306841]
x = [1.1065811525 1.2923404447 1.2004900719 1.1023182776]
x = [0.9368135015 0.8500574843 0.831239914  0.9189940062]
x = [1.0443592245 1.1178843808 1.089151008  1.0445788997]
x = [0.9731063012 0.9343658056 0.9316468014 0.9669158009]
x = [1.0184214585 1.0481614883 1.0388081241 1.0192535562]
x = [0.9885772924 0.9717027268 0.971979774  0.9863750189]
x = [1.0076755408 1.0198642357 1.0166735109 1.0082361203]
x = [0.9951656643 0.9879147419 0.9884162242 0.9943507478]
x = [1.0032073171 1.0082458906 1.0071055273 1.0035005432]
x = [0.9979591779 0.9948689975 0.9951840103 0.997646787 ]
x = [1.0013428486 1.0034375833 1.0030126571 1.0014817059]
x = [0.9991398823 0.9978296481 0.997990254  0.9990167535]

## Bonus

Avoir une boucle qui travaille sur les indices n'est pas satisfaisant. Essayons de comprendre ce qui se passe lorsqu'on utilise la valeur modifiée de `x` dans la boucle `for`. À un instant donnée, lorsque `i == 3` par exemple, les valeurs précédentes, `x[0], x[1], x[2]`, ont déjà été mises à jour. Par contre les valeurs qui suivent, `x[4], x[5]...` sont toujours les anciennes. Donc les valeurs à gauche de la diagonale sont les nouvelles quand celles à droite de la diagonale sont les anciennces.

Cela revient à découper la matrice **A** en une matrice triangulaire inférieure **L** - une matrice triangulaire supérieure strictement **U**. Maintenant la méthode itérative est

$$
{\bf L} \, {\bf x}^{k+1} = {\bf U} \, {\bf x}^k + {\bf b}
$$

Inverser **L** est compliqué mais résoudre ${\bf L} \, {\bf x} = {\bf y}$ est rapide comme on l'a déjà vu. Alors faisons cela :

In [9]:
def Jacobi_bonus(A, b, error=1E-6, verbose=False):
    L = np.tril(A) 
    U = -np.triu(A, k=1) 
    if verbose:
        print(f"L:\n {L}\nU\n {U}\n")
    previous_x = np.zeros(len(b))
    x = np.random.random(len(b))
    err = (error / 10) ** 2
    while np.sum(np.square(x - previous_x))  > err:
        previous_x = x.copy()
        if verbose:
            print(f"x = {x}")
        # on résoud  L x = U x + b  avec L matrice triangulaire inférieure
        y = U @ x + b
        x[0] = y[0] / L[0,0]
        for i in range(1,len(L)):
            x[i] = (y[i] - L[i,:i] @ x[:i]) / L[i,i]
    return x

In [10]:
print('A:\n', A, "\nb:\n", b, "\n")

Jacobi_bonus(A,b, verbose=True)

A:
 [[27  2  1  9]
 [ 5 19  9  5]
 [ 0  6 11  1]
 [ 4  7  1 33]] 
b:
 [39 38 18 45] 

L:
 [[27  0  0  0]
 [ 5 19  0  0]
 [ 0  6 11  0]
 [ 4  7  1 33]]
U
 [[ 0 -2 -1 -9]
 [ 0  0 -9 -5]
 [ 0  0  0 -1]
 [ 0  0  0  0]]

x = [0.1687056829 0.2381905968 0.0198496136 0.921470411 ]
x = [1.118908722  1.4536556743 0.7596905039 0.896638936 ]
x = [1.0097499157 1.1384653267 0.9338699186 0.971450701 ]
x = [1.0017090044 1.0383880108 0.9816564758 0.9922058009]
x = [1.0004338998 1.0106259586 0.9949125861 0.9978475789]
x = [1.0001187884 1.0029449941 0.9985893142 0.9994036537]
x = [1.000032882  1.0008164997 0.9996088498 0.9998346704]
x = [1.0000091154 1.0002263906 0.9998915442 0.9999541594]
x = [1.0000025274 1.000062772  0.999969928  0.9999872896]
x = [1.0000007008 1.000017405  0.9999916618 0.9999964758]
x = [1.0000001943 1.000004826  0.999997688  0.9999990228]
x = [1.0000000539 1.0000013381 0.999999359  0.9999997291]
x = [1.0000000149 1.000000371  0.9999998223 0.9999999249]
x = [1.0000000041 1.0000001029

array([1.0000000011, 1.0000000285, 0.9999999863, 0.9999999942])